# 3. Approche supervisée.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mytools as mt
#import mlflow

## 3.1 Préparation de l'environnement MLflow et chargement des données.

In [4]:
mlflow.set_tracking_uri("file:///home/chakir/Bureau/PROJETS/Projet5/mlruns")

NameError: name 'mlflow' is not defined

Nous allons créer une expérience **approches_supervisées** dans laquelle nous enregistrerons nos runs :

In [ ]:
mlflow.set_experiment('approches_supervisees')

In [2]:
data = pd.read_csv('questions_pretraitees.csv')
data.set_index('Id', inplace=True)
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
48363647,vs eslint pretty use,prettier throw eslint pretty job format hr...,editorconfig eslint pretty code format,110,34657,5
15675173,add time,net test look package android nofollow cla...,java android noclassdeffounderror jodatime lib,19,45741,1
11088303,format d json convert s,find github server function adjacency js f...,graph tree js d json javascript,35,27139,2
40534837,env run kube open kubernete subnet installatio...,fail controller like message compiler ua http ...,docker kube kubernete flannel dns weave,12,26434,2
7281765,youtube video embed iframe,iframe new p way try play youtube video em...,api iframe youtube html javascript,265,1014467,16


In [3]:
data[data.Title.isna()].shape

(20, 6)

In [4]:
condition = data.Title.isna()
data.loc[condition, 'Title'] = data.loc[condition, 'Body']

In [5]:
mt.valeurs_manquantes(data)

,effectif,taux


### TAUX DE COUVERTURE DES TAGS

In [7]:
def tag_coverage(tags_pred, tags_cible):
    coverages = []
    for pred, cible in zip(tags_pred.str.split(), tags_cible.str.split()):
        common_tags = set(pred).intersection(cible)
        coverages.append(len(common_tags) / len(cible) if len(cible) > 0 else 0)
    return sum(coverages) / len(coverages)

In [8]:
tag_coverage(data.Title + ' ' + data.Body, data.Tags)

0.653717333000319

In [9]:
data_val = pd.read_csv('questions_pretraitees_validation.csv')
data_val.set_index('Id', inplace=True)
data_val.shape

(25000, 6)

In [10]:
data_val[data_val.Title.isna()].shape

(26, 6)

In [11]:
condition = data_val.Title.isna()
data_val.loc[condition, 'Title'] = data_val.loc[condition, 'Body']

In [12]:
mt.valeurs_manquantes(data_val)

,effectif,taux


In [24]:
tag_coverage(data_val.Title + ' ' + data_val.Body, data_val.Tags)

0.6513662034631913

### BAG OF WORDS TF-IDF

In [27]:
Tags = pd.read_csv('liste_tags.csv')['0'].tolist()
len(Tags)

7663

In [29]:
Tags = [x for x in Tags if x == x]
len(Tags)

7661

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
# Créer une matrice TF-IDF pour les données textuelles :
vectorizer = TfidfVectorizer(vocabulary=Tags, stop_words='english')
vectorizer.fit(data.Title + ' ' + data.Body + ' ' + data.Tags)

bow_tags = pd.DataFrame(vectorizer.transform(data.Tags).toarray(), index=data.index, columns=Tags)
bow = pd.DataFrame(vectorizer.transform(data.Title + ' ' + data.Body).toarray(), index=data.index, columns=Tags)

In [35]:
bow.head()

,,,,,,,,,,,...,zooming,zope,zpl,zsh,zshrc,zsi,zurb,zuul,zxe,zynq
Id,,,,,,,,,,,,,,,,,,,,,
48363647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15675173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11088303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40534837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7281765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
mt.valeurs_manquantes(bow)

,effectif,taux


In [38]:
bow[(bow == 0).all(axis=1)].shape[0]

0

In [39]:
bow_tags[(bow_tags == 0).all(axis=1)].shape[0]

1

In [40]:
# Calculer la distance cosinus entre deux vecteurs :
def cosine_distance(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

def relative_distance(x, y):
    return np.linalg.norm(x - y)/np.linalg.norm(y)

# Calculer la distance moyenne entre les lignes de deux DataFrames :
def average_distance(df1, df2, metric=cosine_distance):
    distances = []
    for index in df1.index:
            distances.append(metric(df1.loc[index,:], df2.loc[index,:]))
    return np.mean(distances)

In [45]:
condition = (bow_tags == 0).all(axis=1)
average_distance(bow[~condition], bow_tags[~condition])

0.40349571729897166

In [47]:
average_distance(bow[~condition], bow_tags[~condition], metric=relative_distance)

1.0787161083362002

In [49]:
bow_tags_val = pd.DataFrame(vectorizer.transform(data_val.Tags).toarray(), index=data_val.index, columns=Tags)
bow_val = pd.DataFrame(vectorizer.transform(data_val.Title + ' ' + data_val.Body).toarray(), index=data_val.index, columns=Tags)

In [51]:
bow_val[(bow_val == 0).all(axis=1)].shape[0]

0

In [53]:
bow_tags_val[(bow_tags_val == 0).all(axis=1)].shape[0]

0

In [55]:
average_distance(bow_val, bow_tags_val)

0.4045691932542547

In [56]:
average_distance(bow_val, bow_tags_val, metric=relative_distance)

1.0777653991687233

### Word2Vec

In [59]:
from gensim.models import Word2Vec

In [61]:
sentences = [title.split() for title in data.Title] + [question.split() for question in data.Body] + [tags.split() for tags in data.Tags]
embedding = Word2Vec(sentences, vector_size=2, min_count=1, sg=1, window=7)

In [63]:
embedding.wv.most_similar('tensorflow')

[('foolproof', 1.0),
 ('decoder', 1.0),
 ('bufferedreader', 1.0),
 ('uvicorn', 1.0),
 ('scrollintoview', 0.9999999403953552),
 ('gettext', 0.9999997615814209),
 ('xuggle', 0.9999997615814209),
 ('homogenous', 0.9999997019767761),
 ('runtime', 0.9999997019767761),
 ('lateral', 0.9999996423721313)]

In [65]:
def get_vector(embedding, text):
    return np.mean([embedding.wv[word] for word in text.split() if word in embedding.wv], axis=0)

In [67]:
w2v_tags = pd.DataFrame(data.Tags.apply(lambda x : get_vector(embedding, x)).tolist(), index=data.index)
w2v = pd.DataFrame((data.Title + ' ' + data.Body).apply(lambda x: get_vector(embedding, x)).tolist(), index=data.index)

In [69]:
w2v.shape

(25000, 2)

In [71]:
average_distance(w2v, w2v_tags)

0.9771767

In [73]:
average_distance(w2v, w2v_tags, metric=relative_distance)

0.23566364

In [75]:
w2v_tags_val = pd.DataFrame(data_val.Tags.apply(lambda x : get_vector(embedding, x)).tolist(), index=data_val.index)
w2v_val = pd.DataFrame((data_val.Title + ' ' + data_val.Body).apply(lambda x: get_vector(embedding, x)).tolist(), index=data_val.index)

In [77]:
average_distance(w2v_val, w2v_tags_val)

0.97720367

In [79]:
average_distance(w2v_val, w2v_tags_val, metric=relative_distance)

0.23115596

### BERT

In [38]:
import torch
from transformers import BertTokenizer, BertModel # Charger le tokenizer et le modèle BERT

/home/chakir/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [44]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenisation de la phrase
inputs = tokenizer(data.Tags.tolist(), padding=True, truncation=True, return_tensors="pt")

In [ ]:
# Obtenir les embeddings de la couche cachée finale (last_hidden_state)
with torch.no_grad():
    outputs = model(**inputs)

# Récupérer les embeddings de la couche cachée finale
#embeddings = outputs.last_hidden_state

### USE

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

embedding = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

# Obtenir les embeddings des questions
use = embedding((data.Title + ' ' + data.Body).tolist())
use_tags = embedding((data.Tags).tolist())

2024-04-29 16:04:30.947670: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 16:04:32.020503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-29 16:06:31.397626: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19118848 exceeds 10% of free system memory.
2024-04-29 16:06:31.398533: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28678272 exceeds 10% of free system memory.
2024-04-29 16:06:31.433588: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 231600000 exceeds 10% of free system memory.
2024-04-29 16:06:31.665591: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19118848 exceeds 10